In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 1. Multi-Head Attention Layer
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.embed_size = embed_size
        self.head_dim = embed_size // num_heads

        assert self.head_dim * num_heads == embed_size, "Embedding size must be divisible by number of heads"

        self.values = nn.Linear(embed_size, embed_size)
        self.keys = nn.Linear(embed_size, embed_size)
        self.queries = nn.Linear(embed_size, embed_size)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.num_heads different pieces
        values = values.reshape(N, value_len, self.num_heads, self.head_dim).permute(2, 0, 1, 3)
        keys = keys.reshape(N, key_len, self.num_heads, self.head_dim).permute(2, 0, 1, 3)
        queries = query.reshape(N, query_len, self.num_heads, self.head_dim).permute(2, 0, 1, 3)

        # Scaled dot-product attention
        energy = torch.einsum("nhqd,nhkd->nhqk", [queries, keys])  # (num_heads, N, query_len, key_len)
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(energy / (self.embed_size ** (1/2)), dim=-1)

        out = torch.einsum("nhql,nhld->nhqd", [attention, values])  # (num_heads, N, query_len, head_dim)
        out = out.permute(1, 2, 0, 3).reshape(N, query_len, self.num_heads * self.head_dim)

        out = self.fc_out(out)
        return out


# 2. Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, embed_size, max_len=5000):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_len, embed_size)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, embed_size, 2).float() * -(math.log(10000.0) / embed_size))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)

        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]


# 3. Encoder Layer
class EncoderLayer(nn.Module):
    def __init__(self, embed_size, num_heads, dropout):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(embed_size, num_heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, embed_size * 4),
            nn.ReLU(),
            nn.Linear(embed_size * 4, embed_size)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out


# 4. Decoder Layer
class DecoderLayer(nn.Module):
    def __init__(self, embed_size, num_heads, dropout):
        super(DecoderLayer, self).__init__()
        self.attention = MultiHeadAttention(embed_size, num_heads)
        self.encoder_attention = MultiHeadAttention(embed_size, num_heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.norm3 = nn.LayerNorm(embed_size)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, embed_size * 4),
            nn.ReLU(),
            nn.Linear(embed_size * 4, embed_size)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, enc_out, mask):
        attention = self.attention(value, key, query, mask)
        query = self.dropout(self.norm1(attention + query))
        encoder_attention = self.encoder_attention(enc_out, enc_out, query, mask)
        x = self.dropout(self.norm2(encoder_attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm3(forward + x))
        return out


# 5. Transformer Model
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, embed_size, num_heads, num_enc_layers, num_dec_layers, dropout=0.1):
        super(Transformer, self).__init__()
        self.src_word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.tgt_word_embedding = nn.Embedding(tgt_vocab_size, embed_size)
        self.positional_encoding = PositionalEncoding(embed_size)

        self.encoder_layers = nn.ModuleList(
            [EncoderLayer(embed_size, num_heads, dropout) for _ in range(num_enc_layers)]
        )
        self.decoder_layers = nn.ModuleList(
            [DecoderLayer(embed_size, num_heads, dropout) for _ in range(num_dec_layers)]
        )
        self.fc_out = nn.Linear(embed_size, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, tgt, src_mask, tgt_mask):
        src_embed = self.positional_encoding(self.src_word_embedding(src))
        tgt_embed = self.positional_encoding(self.tgt_word_embedding(tgt))

        enc_out = src_embed
        for layer in self.encoder_layers:
            enc_out = layer(enc_out, enc_out, enc_out, src_mask)

        dec_out = tgt_embed
        for layer in self.decoder_layers:
            dec_out = layer(enc_out, enc_out, dec_out, enc_out, tgt_mask)

        output = self.fc_out(dec_out)
        return output


# Example Usage
src_vocab_size = 10000  # Size of the source vocabulary
tgt_vocab_size = 10000  # Size of the target vocabulary
embed_size = 512  # Embedding size
num_heads = 8  # Number of attention heads
num_enc_layers = 6  # Number of encoder layers
num_dec_layers = 6  # Number of decoder layers
dropout = 0.1  # Dropout rate

# Initialize the transformer
model = Transformer(src_vocab_size, tgt_vocab_size, embed_size, num_heads, num_enc_layers, num_dec_layers, dropout)

# Example input (batch_size=2, sequence_length=5)
src = torch.randint(0, src_vocab_size, (2, 5))  # Source sequence
tgt = torch.randint(0, tgt_vocab_size, (2, 5))  # Target sequence
src_mask = None  # Optional mask for source
tgt_mask = None  # Optional mask for target

# Forward pass
output = model(src, tgt, src_mask, tgt_mask)

print(output.shape)  # Output shape will be (batch_size, tgt_sequence_length, tgt_vocab_size)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

# 1. Multi-Head Attention Layer
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.embed_size = embed_size
        self.head_dim = embed_size // num_heads

        assert self.head_dim * num_heads == embed_size, "Embedding size must be divisible by number of heads"

        self.values = nn.Linear(embed_size, embed_size)
        self.keys = nn.Linear(embed_size, embed_size)
        self.queries = nn.Linear(embed_size, embed_size)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, values, keys, query, mask=None):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.num_heads different pieces
        values = values.reshape(N, value_len, self.num_heads, self.head_dim).permute(2, 0, 1, 3)
        keys = keys.reshape(N, key_len, self.num_heads, self.head_dim).permute(2, 0, 1, 3)
        queries = query.reshape(N, query_len, self.num_heads, self.head_dim).permute(2, 0, 1, 3)

        # Scaled dot-product attention
        energy = torch.einsum("nhqd,nhkd->nhqk", [queries, keys])  # (num_heads, N, query_len, key_len)
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(energy / (self.embed_size ** (1/2)), dim=-1)

        out = torch.einsum("nhql,nhld->nhqd", [attention, values])  # (num_heads, N, query_len, head_dim)
        out = out.permute(1, 2, 0, 3).reshape(N, query_len, self.num_heads * self.head_dim)

        out = self.fc_out(out)
        return out


# 2. Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, embed_size, max_len=5000):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_len, embed_size)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, embed_size, 2).float() * -(math.log(10000.0) / embed_size))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)

        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]


# 3. Encoder Layer
class EncoderLayer(nn.Module):
    def __init__(self, embed_size, num_heads, dropout):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(embed_size, num_heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, embed_size * 4),
            nn.ReLU(),
            nn.Linear(embed_size * 4, embed_size)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out


# 4. Decoder Layer
class DecoderLayer(nn.Module):
    def __init__(self, embed_size, num_heads, dropout):
        super(DecoderLayer, self).__init__()
        self.attention = MultiHeadAttention(embed_size, num_heads)
        self.encoder_attention = MultiHeadAttention(embed_size, num_heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.norm3 = nn.LayerNorm(embed_size)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, embed_size * 4),
            nn.ReLU(),
            nn.Linear(embed_size * 4, embed_size)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, enc_out, mask):
        attention = self.attention(value, key, query, mask)
        query = self.dropout(self.norm1(attention + query))
        encoder_attention = self.encoder_attention(enc_out, enc_out, query, mask)
        x = self.dropout(self.norm2(encoder_attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm3(forward + x))
        return out


# 5. Transformer Model
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, embed_size, num_heads, num_enc_layers, num_dec_layers, dropout=0.1):
        super(Transformer, self).__init__()
        self.src_word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.tgt_word_embedding = nn.Embedding(tgt_vocab_size, embed_size)
        self.positional_encoding = PositionalEncoding(embed_size)

        self.encoder_layers = nn.ModuleList(
            [EncoderLayer(embed_size, num_heads, dropout) for _ in range(num_enc_layers)]
        )
        self.decoder_layers = nn.ModuleList(
            [DecoderLayer(embed_size, num_heads, dropout) for _ in range(num_dec_layers)]
        )
        self.fc_out = nn.Linear(embed_size, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, tgt, src_mask, tgt_mask):
        src_embed = self.positional_encoding(self.src_word_embedding(src))
        tgt_embed = self.positional_encoding(self.tgt_word_embedding(tgt))

        enc_out = src_embed
        for layer in self.encoder_layers:
            enc_out = layer(enc_out, enc_out, enc_out, src_mask)

        dec_out = tgt_embed
        for layer in self.decoder_layers:
            dec_out = layer(enc_out, enc_out, dec_out, enc_out, tgt_mask)

        output = self.fc_out(dec_out)
        return output


# Example usage of the Transformer for Language Translation:

# Assume we have a source and target vocabulary size (for simplicity, set them to 10,000 here).
src_vocab_size = 10000  # Example source vocabulary size (e.g., English)
tgt_vocab_size = 10000  # Example target vocabulary size (e.g., French)
embed_size = 512  # Embedding size
num_heads = 8  # Number of attention heads
num_enc_layers = 6  # Number of encoder layers
num_dec_layers = 6  # Number of decoder layers
dropout = 0.1  # Dropout rate

# Initialize the Transformer model
model = Transformer(src_vocab_size, tgt_vocab_size, embed_size, num_heads, num_enc_layers, num_dec_layers, dropout)

# Example input data
src = torch.randint(0, src_vocab_size, (2, 10))  # Example source sequence (batch_size=2, seq_length=10)
tgt = torch.randint(0, tgt_vocab_size, (2, 8))  # Example target sequence (batch_size=2, seq_length=8)

# Example masks (can be used for padding or causal masking)
src_mask = None  # Assuming no padding mask for simplicity
tgt_mask = None  # Assuming no padding mask for simplicity

# Forward pass (translation)
output = model(src, tgt, src_mask, tgt_mask)

print(output.shape)  # Output shape: (batch_size, tgt_seq_len, tgt_vocab_size)


torch.Size([2, 8, 10000])
